In [3]:
import os
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

# Multi-Agent - Orchestrate Teams of Agents

In [9]:
from openai import OpenAI
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager, register_function

In [28]:
import json
from typing import List
def generate_and_save_possible_candidates(query: str)->List[str]:

    client = OpenAI()
    client.api_key=os.environ.get("OPENAI_API_KEY")
    response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{query}"},
        ]
    )
    chat_content = response.choices[0].message.content
    # chat_response = json.loads(
    #     chat_content
    # )
    print(chat_content)
    return chat_content


In [41]:
organization_director = AssistantAgent(
    name="organization_director",
    llm_config=llm_config,
    system_message="""You are a helpful assistant that is a highly skilled AI conference organization director. Given a general description of a conference, 
    your goal is to provide a detailed and creative plan that develops the provided description into a very high quality, inclusive, accessible AI conference for Women in Tech community. 
    Your plan should include details on the intended location, the description web page, cost items, the number of speakers, field of speakers, marketing materials etc. In 30 items would be sufficient. """,
)
financial_director = AssistantAgent(
    name="financial_director",
    llm_config=llm_config,
    system_message="""You are a helpful assistant that is a highly skilled financial director for an AI conference. Given a general description of a conference
      provided by a conference organizer, your goal is to create highly detailed conceptual cost items that will be required to do the AI conference.
      Each cost items should have name, the reason why it's needed, and its cost for today. You should create a csv file whose name is diva_budget.csv that has all these fields.""",
)

speaker_admin = AssistantAgent(
    name="speaker_admin",
    llm_config=llm_config,
    system_message="""You are a helpful assistant that is a highly skilled speaker admin for an AI conference. Given a general description of a conference
      provided by a conference organizer, your goal is to list the candidate speakers and their talk titles etc. Make sure the majority of speakers are from the country that is provided by a conference organizer. 
      You should generate a PDF file that will include a time table that matches with speaker's name, speaker's biography, talk title, and their talk abstract. It should be stored in speakers.pdf""",
)

marketing_admin = AssistantAgent(
    name="marketing_admin",
    llm_config=llm_config,
    system_message="""You are a helpful assistant that is a highly skilled marketing admin for an AI conference. 
                                 Given a general description of the conference provided by a conference organizer and the list of speakers provided by speaker comittee, 
                                 your goal is to create highly detailed social media posts to introduce speakers and to attract the audience. 
                                 Your posts should be detailed and specifically created for different social medias.""",
)

In [42]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config={"work_dir": "data/scratch", "use_docker": False},
    human_input_mode="NEVER",
    is_termination_msg=lambda x: (
        x.get("content", "").rstrip().endswith("TERMINATE")
        if x.get("content")
        else False
    ),
)

register_function(
    generate_and_save_possible_candidates,
    caller=speaker_admin,
    executor=user_proxy,
    name="generate_and_save_possible_candidates",
    description="Generate candidate speakers list using GPT3.5",
)

groupchat = GroupChat(agents=[organization_director,financial_director,speaker_admin,marketing_admin,user_proxy], messages=[], speaker_selection_method="round_robin")
groupchat_manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [43]:
user_proxy.initiate_chat(
    groupchat_manager,
    message="Plan a 1 day AI conference for Women in Tech Community with the theme of 'accessibility and inclusiveness' in Istanbul the name of the conference will be Diva:Dive Into AI 2025.",
)

user_proxy (to chat_manager):

Plan a 1 day AI conference for Women in Tech Community with the theme of 'accessibility and inclusiveness' in Istanbul the name of the conference will be Diva:Dive Into AI 2025.

--------------------------------------------------------------------------------

Next speaker: organization_director

organization_director (to chat_manager):

Certainly! I'll create a detailed plan for the "Diva: Dive Into AI 2025" conference, focusing on accessibility and inclusiveness for the Women in Tech community in Istanbul. Here's a comprehensive 30-item plan:

1. Location: Istanbul Congress Center, chosen for its central location and accessibility features.

2. Date: September 15, 2025 (chosen to avoid religious holidays and summer vacation period)

3. Conference Website: Create a fully accessible website with features like screen reader compatibility, high contrast options, and easy navigation.

4. Registration System: Implement an inclusive online registration system 

ChatResult(chat_id=None, chat_history=[{'content': "Plan a 1 day AI conference for Women in Tech Community with the theme of 'accessibility and inclusiveness' in Istanbul the name of the conference will be Diva:Dive Into AI 2025.", 'role': 'assistant'}, {'content': 'Certainly! I\'ll create a detailed plan for the "Diva: Dive Into AI 2025" conference, focusing on accessibility and inclusiveness for the Women in Tech community in Istanbul. Here\'s a comprehensive 30-item plan:\n\n1. Location: Istanbul Congress Center, chosen for its central location and accessibility features.\n\n2. Date: September 15, 2025 (chosen to avoid religious holidays and summer vacation period)\n\n3. Conference Website: Create a fully accessible website with features like screen reader compatibility, high contrast options, and easy navigation.\n\n4. Registration System: Implement an inclusive online registration system with options for dietary requirements, accessibility needs, and preferred pronouns.\n\n5. Tick